In [21]:
# import all necessary libraries
!pip install mysql-connector-python
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import sklearn
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import time
import json
import re
import math
import pickle
from datetime import datetime
from datetime import timedelta
import requests
from requests.auth import AuthBase
import config
import tools
import mysql.connector
import urllib.parse
from mysql.connector import errorcode
import sqlite3
%matplotlib inline
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [22]:
# class definition for BearerTokenAuth, inherits from AuthBase
class BearerTokenAuth(AuthBase):
    """class which handles bearer token requests and authentification"""
    def __init__(self, consumer_key, consumer_secret):
        """initializes a BearerTokenAuth object"""
        self.bearer_token_url = "https://api.twitter.com/oauth2/token"
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.bearer_token = self.get_bearer_token()

    def get_bearer_token(self):
        """requests bearer token"""
        response = requests.post(
            self.bearer_token_url,
            auth=(self.consumer_key, self.consumer_secret),
            data={'grant_type': 'client_credentials'},
            headers={'User-Agent': 'LabsRecentSearchQuickStartPython'})

        if response.status_code is not 200:
            raise Exception("Cannot get a Bearer token (HTTP %d): %s" %
                            (response.status_code, response.text))

        body = response.json()
        return body['access_token']

    def __call__(self, r):
        """sets headers"""
        r.headers['Authorization'] = f"Bearer %s" % self.bearer_token
        r.headers['User-Agent'] = 'LabsRecentSearchQuickStartPython'
        return r

In [23]:
# define a mysql connection to aws db
def aws_connect(db_name=None):
    """function which will connect us to aws database instance of our choosing"""
    connection = mysql.connector.connect(
    host = config.AWS_ENDPOINT,
    user = config.AWS_USER,
    passwd = config.AWS_PASSWORD,
    port = config.AWS_PORT,
    database = db_name)
    return connection

# create a connection with aws db
cnx = aws_connect()

# create a cursor over the mysql connection
cursor = cnx.cursor()

# function to create our AWS database
def create_database(cursor, database):
    """creates an aws instance"""
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
        
# name of our database
db_name = '$AMZN'

# check to see if the database already exists, and if it doesn't, create it
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)
        
# template for our db tables
TABLES = {}
TABLES['tweet_time_price1'] = (
    "CREATE TABLE tweet_time_price1 ("
    "  tweet varchar(250) NOT NULL,"
    "  id varchar(50) NOT NULL,"
    "  datetime datetime NOT NULL,"
    "  price varchar(10) NOT NULL"
    ") ENGINE=InnoDB")

# create tables according to above template
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

# close connections to our cursors
cursor.close()
cnx.close()
# reopen a mysql connection
cnx = aws_connect('$AMZN')
# recreate a cursor over the mysql connection
cursor = cnx.cursor()
# insert statement to add a tweet
add_tweet = ("INSERT INTO tweet_time_price1"
               "(tweet, id, datetime, price)"
               "VALUES (%s, %s, %s, %s)")

Creating table tweet_time_price1: already exists.


In [24]:
# time intervals over which we'll query our tweets
initial_time = '2020-05-19 18:04:00' # adjusted to last break of loop
final_time = '2020-05-21 23:00:00'
intervals = list(pd.date_range(initial_time, final_time, freq='T'))
intervals1 = [str(i).split() for i in intervals]
intervals2 = [i[0]+"T"+i[1]+"Z" for i in intervals1]
intervals2

['2020-05-19T18:04:00Z',
 '2020-05-19T18:05:00Z',
 '2020-05-19T18:06:00Z',
 '2020-05-19T18:07:00Z',
 '2020-05-19T18:08:00Z',
 '2020-05-19T18:09:00Z',
 '2020-05-19T18:10:00Z',
 '2020-05-19T18:11:00Z',
 '2020-05-19T18:12:00Z',
 '2020-05-19T18:13:00Z',
 '2020-05-19T18:14:00Z',
 '2020-05-19T18:15:00Z',
 '2020-05-19T18:16:00Z',
 '2020-05-19T18:17:00Z',
 '2020-05-19T18:18:00Z',
 '2020-05-19T18:19:00Z',
 '2020-05-19T18:20:00Z',
 '2020-05-19T18:21:00Z',
 '2020-05-19T18:22:00Z',
 '2020-05-19T18:23:00Z',
 '2020-05-19T18:24:00Z',
 '2020-05-19T18:25:00Z',
 '2020-05-19T18:26:00Z',
 '2020-05-19T18:27:00Z',
 '2020-05-19T18:28:00Z',
 '2020-05-19T18:29:00Z',
 '2020-05-19T18:30:00Z',
 '2020-05-19T18:31:00Z',
 '2020-05-19T18:32:00Z',
 '2020-05-19T18:33:00Z',
 '2020-05-19T18:34:00Z',
 '2020-05-19T18:35:00Z',
 '2020-05-19T18:36:00Z',
 '2020-05-19T18:37:00Z',
 '2020-05-19T18:38:00Z',
 '2020-05-19T18:39:00Z',
 '2020-05-19T18:40:00Z',
 '2020-05-19T18:41:00Z',
 '2020-05-19T18:42:00Z',
 '2020-05-19T18:43:00Z',


In [25]:
# constant search parameters
query = urllib.parse.quote(""""$AMZN" lang:en""")
max_results = "100"
headers = {"Accept-Encoding": "gzip"}

# Script starts here
for i in range(len(intervals2)-1):
    
    start = intervals2[i]
    end =  intervals2[i+1]
    url = f"https://api.twitter.com/labs/2/tweets/search?query={query}&max_results={max_results}&start_time={start}&end_time={end}"
    
    # first page of calls
    response = requests.get(url, auth=BearerTokenAuth(config.TWITTER_API_KEY, config.TWITTER_API_SECRET_KEY), headers=headers)
    if response.status_code is not 200:
        raise Exception(f"Request returned an error: %s, %s" % (response.status_code, response.text))
    parsed = json.loads(response.text)
    pretty_print = json.dumps(parsed, indent=2, sort_keys=True)
    print(pretty_print)
    # adds our parsed results to our database
    try:
        for j in parsed['data']:
            cursor.execute(add_tweet, (j['text'], j['id'], intervals2[i+1], 'null'))
            cnx.commit()
    except:
        continue
    time.sleep(5)

Exception: Request returned an error: 503, {"title":"Service Unavailable","type":"about:blank","status":503,"detail":"Service Unavailable"}

In [7]:
intervals2[i]


'2020-05-19T18:04:00Z'

In [ ]:
# recall our tweets from our aws database
cursor.execute("""SELECT tweet, id, datetime, price FROM $AMZN.tweet_time_price1 ORDER BY datetime ASC""")
# store the tweets in a dataframe
df1 = pd.DataFrame(cursor.fetchall())
df1.columns = [x[0] for x in cursor.description]
df1

In [ ]:
# pickle our dataframe of tweets to save the work we've done
with open("AMZN_tweets_df.pkl", 'wb') as f:
    pickle.dump(df1, f)

In [ ]:
# reload our pickled tweets into our df; this serves as a checkpoint
with open("AMZN_tweets_df.pkl", 'rb') as f:
    df1 = pickle.load(f)
df1

In [ ]:
# sort tweets
df1.sort_values("tweet", inplace = True) 
# drop duplicte values 
df1.drop_duplicates(subset ="tweet", keep = 'first', inplace = True)
# resort our de-duped tweets by datetime
df1.sort_values("datetime", inplace = True)
df1

In [ ]:
# instantiate a VADER sentiment analyzer
analyser = SentimentIntensityAnalyzer()
# get sentiment scores, store compound sentiment score as our sentiment column
df1['sentiment'] = [analyser.polarity_scores(i) for i in df1['tweet']]
df1['compound_sentiment'] = [i['compound'] for i in df1['sentiment']]
df1['positive_sentiment'] = [i['pos'] for i in df1['sentiment']]
df1['negative_sentiment'] = [i['neg'] for i in df1['sentiment']]
df1['neutral_sentiment'] = [i['neu'] for i in df1['sentiment']]
# drop the column containing sentiment dictionary object
df1.drop(columns=['sentiment'], inplace=True)
df1

In [ ]:
# quick visual of cyclical tweet volume
sns.distplot(df1.index)

In [ ]:
# save our tweets for pickle checkpoint
with open("AMZN_tweets_df.pkl", 'wb') as f:
    pickle.dump(df1, f)